# ExperimentalFactor

lamindb provides access to the following public protein ontologies through [lnschema-bionty](https://github.com/laminlabs/lnschema-bionty):

1. [Experimental Factor Ontology](https://www.ebi.ac.uk/ols/ontologies/efo)

Here we show how to access and search ExperimentalFactor ontologies to standardize new data.

## Setup

In [ ]:
!lamin init --storage ./test-experimentalfactor --schema bionty

In [ ]:
import lnschema_bionty as lb
import pandas as pd

## PublicOntology objects

Let us create a public ontology accessor with {meth}`~docs:lnschema_bionty.dev.BioRegistry.public`, which chooses a default public ontology source from {class}`~docs:lnschema_bionty.PublicSource`.
It's a [PublicOntology](https://lamin.ai/docs/lnschema_bionty.dev.publicontology) object, which you can think about as a public registry:

In [ ]:
experimentalfactors = lb.ExperimentalFactor.public(organism="all")
experimentalfactors

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = experimentalfactors.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `public.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = experimentalfactors.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.sequencer

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["sequencer"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = experimentalfactors.lookup(experimentalfactors.ontology_id)

In [ ]:
lookup.efo_0003739

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
experimentalfactors.search("single-cell rna seq").head(3)

By default, search also covers synonyms:

In [ ]:
experimentalfactors.search("single-cell RNA-seq").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
experimentalfactors.search("single-cell RNA-seq", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
experimentalfactors.search(
    "assay which analysis the transcriptome of a biological sample",
    field=experimentalfactors.definition,
).head()

## Standardize ExperimentalFactor identifiers

Let us generate a `DataFrame` that stores a number of ExperimentalFactor identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
    index=[
        "EFO:0011021"
        "EFO:1002050"
        "EFO:1002047"
        "EFO:1002049"
        "This experimentalfactor does not exist",
    ]
)
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = experimentalfactors.validate(df_orig.index, experimentalfactors.name)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.PublicSource.filter(entity="ExperimentalFactor").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
public_source = lb.PublicSource.filter(
    source="efo", version="3.48.0", organism="all"
).one()
experimentalfactors= lb.ExperimentalFactor.public(public_source=public_source)
experimentalfactors

The currently used ontologies can be displayed using:

In [ ]:
lb.PublicSource.filter(currently_used=True).df()

In [ ]:
!lamin delete --force test-experimentalfactor
!rm -r test-experimentalfactor